In [32]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [10]:
df = pd.read_csv("/Users/naushin/Downloads/amazon.csv")

In [11]:
df.head(10)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
5,B08Y1TFSP6,pTron Solero TB301 3A Type-C Data and Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹149,"₹1,000",85%,3.9,"24,871",Fast Charging & Data Sync: Solero TB301 Type-C...,"AEQ2YMXSZWEOHK2EHTNLOS56YTZQ,AGRVINWECNY7323CW...","Jayesh,Rajesh k.,Soopy,amazon customer,Aman,Sh...","R7S8ANNSDPR40,R3CLZFLHVJU26P,RFF7U7MPQFUGR,R1M...","It's pretty good,Average quality,very good and...","It's a good product.,Like,Very good item stron...",https://m.media-amazon.com/images/I/31wOPjcSxl...,https://www.amazon.in/Solero-TB301-Charging-48...
6,B08WRWPM22,"boAt Micro USB 55 Tangle-free, Sturdy Micro US...",Computers&Accessories|Accessories&Peripherals|...,₹176.63,₹499,65%,4.1,"15,188",It Ensures High Speed Transmission And Chargin...,"AG7C6DAADCTRQJG2BRS3RIKDT52Q,AFU7BOMPVJ7Q3TTA4...","Vivek kumar,Amazon Customer,SARTHAK,Chiranjeev...","R8E73K2KWJRDS,RSD0JTIIWQQL8,R64CRSTE9SLW1,R2FR...","Long durable.,good,Does not charge Lenovo m8 t...",Build quality is good and it is comes with 2 y...,https://m.media-amazon.com/images/W/WEBP_4

In [15]:
# Cleaning
def clean_price(x):
    """Remove ₹, commas, convert to float."""
    if pd.isna(x):
        return None
    if isinstance(x, str):
        x = x.replace("₹", "").replace(",", "").strip()
    try:
        return float(x)
    except:
        return None

def clean_discount(x):
    """Remove % and convert to float."""
    if pd.isna(x):
        return None
    if isinstance(x, str):
        x = x.replace("%", "").strip()
    try:
        return float(x)
    except:
        return None

df["actual_price"] = df["actual_price"].apply(clean_price)
df["discounted_price"] = df["discounted_price"].apply(clean_price)
df["discount_percentage"] = df["discount_percentage"].apply(clean_discount)


In [21]:
# INR to USD (1 USD = 83 INR)
INR_TO_USD = 1 / 83  

# Convert INR to USD
df["actual_price_usd"] = df["actual_price"] * INR_TO_USD
df["discounted_price_usd"] = df["discounted_price"] * INR_TO_USD

# Round for clean presentation
df["actual_price_usd"] = df["actual_price_usd"].round(2)
df["discounted_price_usd"] = df["discounted_price_usd"].round(2)

df['actual_price_usd'].head()

0    0.145651
1    0.046253
2    0.251675
3    0.092639
4    0.052880
Name: actual_price_usd, dtype: float64

In [17]:
#clean rating count
df["rating_count"] = (
    df["rating_count"]
    .astype(str)
    .str.replace(",", "")
    .astype(float)
)

df['rating_count'].head()

0    24269.0
1    43994.0
2     7928.0
3    94363.0
4    16905.0
Name: rating_count, dtype: float64

In [18]:
# Drop rows with missing key values
df = df.dropna(subset=["actual_price", "discounted_price"])

In [28]:
# Replace weird characters with NaN
df = df.replace({'|': None, '': None, ' ': None})

def safe_float(x):
    try:
        return float(x)
    except:
        return None

In [22]:
# Add engineered features
df["title_length"] = df["product_name"].astype(str).apply(lambda x: len(x.split()))
df["feature_count"] = df["about_product"].astype(str).apply(lambda x: len(x.split("|")))

In [34]:
# Predict discount percentage

categorical_cols = ["category"]
numeric_cols_A = ["actual_price_usd", "rating", "rating_count", "title_length", "feature_count"]

X_A = df[categorical_cols + numeric_cols_A]
y_A = df["discount_percentage"]

# Preprocess categories
preprocess_A = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", SimpleImputer(strategy="median"), numeric_cols_A)
    ]
)

# Pipeline with Random Forest
model_A = Pipeline(steps=[
    ("preprocess", preprocess_A),
    ("rf", RandomForestRegressor(n_estimators=300, random_state=42))
])

# Train/test split
XA_train, XA_test, yA_train, yA_test = train_test_split(X_A, y_A, test_size=0.2, random_state=42)

# Fit model
model_A.fit(XA_train, yA_train)

# Predict
yA_pred = model_A.predict(XA_test)

# Evaluate
print("MODEL A: Predict Discount Percentage")
print("R2:", r2_score(yA_test, yA_pred))
print("RMSE:", np.sqrt(mean_squared_error(yA_test, yA_pred)))


MODEL A: Predict Discount Percentage
R2: 0.5376134160736512
RMSE: 13.44173004546119


In [33]:
# Predict discounted price
numeric_cols_B = ["actual_price_usd", "rating", "rating_count", "title_length", "feature_count"]

# Make sure categorical columns are properly converted to string type
X_B = df[categorical_cols + numeric_cols_B].copy()
for col in categorical_cols:
    X_B[col] = X_B[col].astype(str)  # Convert categorical columns to string type

y_B = df["discounted_price_usd"]

# Preprocess - Add sparse_threshold=0 to ensure dense output
preprocess_B = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", SimpleImputer(strategy="median"), numeric_cols_B)
    ]
)

model_B = Pipeline(steps=[
    ("preprocess", preprocess_B),
    ("rf", RandomForestRegressor(n_estimators=300, random_state=42))
])

# Train/test split
XB_train, XB_test, yB_train, yB_test = train_test_split(X_B, y_B, test_size=0.2, random_state=42)

# Fit
model_B.fit(XB_train, yB_train)

# Predict
yB_pred = model_B.predict(XB_test)

# Evaluate
print("\nMODEL B: Predict Discounted Price")
print("R2:", r2_score(yB_test, yB_pred))
print("RMSE:", np.sqrt(mean_squared_error(yB_test, yB_pred)))


MODEL B: Predict Discounted Price
R2: 0.9638261508639584
RMSE: 0.17926162179331212


In [35]:
# PREDICTION FUNCTIONS FOR THE DASHBOARD

def predict_discount_percentage(category, actual_price_usd, rating, rating_count, title_length, feature_count):
    input_df = pd.DataFrame({
        "category": [category],
        "actual_price_usd": [actual_price_usd],
        "rating": [rating],
        "rating_count": [rating_count],
        "title_length": [title_length],
        "feature_count": [feature_count]
    })
    return model_A.predict(input_df)[0]


def predict_discounted_price(category, actual_price_usd, rating, rating_count, title_length, feature_count):
    input_df = pd.DataFrame({
        "category": [category],
        "actual_price_usd": [actual_price_usd],
        "rating": [rating],
        "rating_count": [rating_count],
        "title_length": [title_length],
        "feature_count": [feature_count]
    })
    return model_B.predict(input_df)[0]

In [36]:
test_discount_percent = predict_discount_percentage(
    category="Computers&Accessories",
    actual_price_usd=10.99,
    rating=4.2,
    rating_count=1500,
    title_length=8,
    feature_count=5
)

test_discount_price = predict_discounted_price(
    category="Computers&Accessories",
    actual_price_usd=10.99,
    rating=4.2,
    rating_count=1500,
    title_length=8,
    feature_count=5
)

print("Predicted Discount %:", round(test_discount_percent, 2))
print("Predicted Discounted Price (USD):", round(test_discount_price, 2))

Predicted Discount %: 38.64
Predicted Discounted Price (USD): 6.84
